In [24]:
from pathlib import Path
DATA_DIR     = Path("/home/saleh/KinForm-1")
ROOT        = Path("/home/saleh/KinForm-1")
EITLEM_DIR  = ROOT / "data/EITLEM_data"
RAW_DLKCAT     = DATA_DIR / "data/dlkcat_raw.json"
EITLEM_JSON   = EITLEM_DIR / "KCAT/kcat_data.json"
SEQ_LOOKUP   = DATA_DIR / "results/sequence_id_to_sequence.pkl"
import pickle
import json
import pandas as pd
# write fasta with all unique sequences

# read DLKCAT
seq_look_up = pickle.load(SEQ_LOOKUP.open("rb"))
seq_to_id = {v: k for k, v in seq_look_up.items()}

In [28]:
raw = [
    d
    for d in json.loads(RAW_DLKCAT.read_text())
    if len(d["Sequence"]) <= 1_499 and float(d["Value"]) > 0 and "." not in d["Smiles"]
]
sequences = [d["Sequence"] for d in raw]
seq_ids_dlkcat = [seq_to_id[s] for s in sequences]


In [25]:
#read eitlem
with EITLEM_JSON.open() as fp:
    raw = json.load(fp)
valid = [(i, r) for i, r in enumerate(raw) if len(r["sequence"]) <= 1499 and float(r["value"]) > 0]
orig_idx = [i for i, _ in valid]
seqs = [r["sequence"] for _, r in valid]
seq_ids_eitlem = [seq_to_id[s] for s in seqs]

In [26]:
#read KM data
with open("/home/saleh/KinForm-1/data/KM_data_raw.json", 'r') as fp:
    raw = json.load(fp)

raw = [d for d in raw if len(d["Sequence"]) <= 1499 and "." not in d["smiles"]]
sequences = [d["Sequence"] for d in raw]
seq_ids_km = [seq_to_id[s] for s in sequences]

In [32]:
seq_id_all = list(set(seq_ids_dlkcat + seq_ids_eitlem + seq_ids_km))
# write fasta with all unique sequences
with open(DATA_DIR / "data/unique_sequences.fasta", 'w') as fp:
    for seq_id in seq_id_all:
        seq = seq_look_up[seq_id]
        seq_id_nospace = seq_id.replace(" ", "_")
        fp.write(f">{seq_id_nospace}\n{seq}\n")

In [42]:
import pandas as pd
path = '/home/saleh/KinForm-1/results/catalytic_sites/unique_sequences_results.pkl'
df = pd.read_pickle(path)

In [ ]:
# rename 'label' column to 'sequence_id'
# rename sequence ids from 'Sequence_{id}' to 'Sequence {id}'
df.rename(columns={'label': 'sequence_id'}, inplace=True)
df['sequence_id'] = df['sequence_id'].str.replace('_', ' ', regex=False)

In [70]:
df['all_AS_probs'] = df['all_AS_probs'].apply(str)
df = df[['sequence_id', 'all_AS_probs']]
# Save entire dataframe
df.to_csv("/home/saleh/KinForm-1/results/catalytic_sites/cat_sites.csv", index=False)

/tmp/ipykernel_3169646/4245417682.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['all_AS_probs'] = df['all_AS_probs'].apply(str)


In [71]:
#read csv
df = pd.read_csv("/home/saleh/KinForm-1/results/catalytic_sites/cat_sites.csv")
# convert 'all_AS_probs' column from string representation of list to actual list
import ast
df['all_AS_probs'] = df['all_AS_probs'].apply(ast.literal_eval)
df

,sequence_id,all_AS_probs
0,Sequence 26683,"[0.0019, 0.0019, 0.0018, 0.0018, 0.0023, 0.002..."
1,Seqence 614139,"[0.0019, 0.0019, 0.0018, 0.0018, 0.0018, 0.001..."
2,Sequence 25920,"[0.002, 0.0019, 0.0016, 0.0018, 0.0018, 0.0018..."
3,Sequence 25481,"[0.0019, 0.0019, 0.0018, 0.0018, 0.0018, 0.001..."
4,Sequence 48,"[0.002, 0.0019, 0.0018, 0.0018, 0.0018, 0.0018..."
...,...,...
19512,Sequence 13153,"[0.0019, 0.0019, 0.0018, 0.0018, 0.0018, 0.001..."
19513,Sequence 26044,"[0.0019, 0.0019, 0.0018, 0.0018, 0.0018, 0.001..."
19514,Sequence 2260,"[0.0019, 0.0019, 0.0018, 0.0018, 0.0018, 0.001..."
19515,Sequence 25002,"[0.0019, 0.0019, 0.0018, 0.0018, 0.0018, 0.001..."


In [77]:
path = '/home/saleh/KinForm-1/results/binding_sites/prediction_2.tsv'
df = pd.read_csv(path, sep='\t')

In [79]:
df ['Pred_BS_Scores'] = df['Pred_BS_Scores'].apply(ast.literal_eval)